In [1]:
from time import perf_counter as tick
import os, re
import numpy as np
import matplotlib.pyplot as mp
import scipy.interpolate as sp
import spec_funcs as sf

##### Import Raw Spectroscopy Data

###### Load raw data from csv file (delimiter is not a problem) and sort into groups - polarisations -> reference / spectrum


In [8]:
# change to gui at later date
polarisations = ["143", "188"]

bg_path = 'C:\\Users\\sk88\Desktop\\Analyse_test'

file_list = []
folder_list = []
holder = 0

for root, dirs, files in os.walk(bg_path):
    if holder == 1:
        folder_list.append(root)
        temp =[]
        for file in files:
            if(file.endswith((".csv", ".txt"))):
                if "notes" in file or "setup" in file:
                    continue
                else:
                    temp.append(file)
        file_list.append(temp)
    else:
        holder = 1

bg_wavelength = []
bg_I = []
bg_wave_sets = []
bg_I_sets = []
wavelength = []
wavenumber = []
I = []
wave_sets = []
I_sets = []

for index, polarisation in enumerate(polarisations):
    for loc in file_list:
        bg_temp_wave = []
        bg_temp_I = []
        temp_wave = []
        temp_I = []
        if polarisations[index] in loc:
            if "ref" in loc or "reference" in loc:
                with open(loc, 'r', newline='') as raw_file:
                    for row in raw_file:
                        if sf.check_str(row) == True:
                            temp = re.split('\t|,|;', row)
                            bg_temp_wave.append(float(temp[0]))
                            bg_temp_I.append(float(temp[1]))
                    raw_file.close()
                    bg_wavelength.append(bg_temp_wave)
                    bg_I.append(bg_temp_I)
            else:
                with open(loc, 'r', newline='') as raw_file:
                    for row in raw_file:
                        if sf.check_str(row) == True:
                            temp = re.split('\t|,|;', row)
                            temp_wave.append(float(temp[0]))
                            temp_I.append(float(temp[1]))
                    raw_file.close()
                    wavelength.append(temp_wave)
                    I.append(temp_I)
bg_wave_sets.append(bg_wavelength)
bg_I_sets.append(bg_I)
wave_sets.append(wavelength)
I_sets.append(I)

#####  Calculate OD

###### Generate interplolated background data to match raw absorption data length and then calculate Optical Depth

In [ ]:
I_0 = []
OD = []

for index in range(len(bg_wavelength)):
    spline_coeffs = sp.splrep(bg_wavelength[index], bg_I[index], k=2)
    for index in range(len(wavelength)):
        I_0.append(sp.splev(wavelength[index], spline_coeffs))
        OD.append(np.log(I_0[index]/I[index]))

zoom = 1
start = []
stop = []

if zoom == 1:

    lower = 550
    upper = 650

    for values in wavelength:
        for index, value in enumerate(values):
            if value <= lower:
                temp_start = index
            if value <= upper:
                temp_stop = index
        start.append(temp_start)
        stop.append(temp_stop)
elif zoom != 1:
    for index in range(len(wavelength)):
        start.append(0)
        stop.append(-1)

OD_norm = OD / np.amax(OD)

##### Print Raw Output

###### Plot individual spectroscopy data

In [ ]:
fig_1, ax_1 = mp.subplots(figsize=(8, 5))
ax_1.set_title(file_name[0])
ax_1.set(xlabel='Wavelength (nm)', ylabel='OD Normalised')
sec_ax = ax_1.secondary_xaxis('top', functions= (lambda x: 1e7 / x, lambda x: 1e7 / x))
sec_ax.set_xlabel('Wavenumber (cm$^{-1}$)')
ax_1.grid(True)
ax_1.grid(True, color='silver', linewidth=0.5)

fig_2, ax_2 = mp.subplots(figsize=(8, 5))
ax_2.set_title('Pr$^{3+}$:YVO')
ax_2.set(xlabel='Wavelength (nm)', ylabel='Absorption')
sec_ax = ax_2.secondary_xaxis('top', functions= (lambda x: 1e7 / x, lambda x: 1e7 / x))
sec_ax.set_xlabel('Wavenumber (cm$^{-1}$)', fontsize=12)
ax_2.grid(True)
ax_2.grid(True, color='silver', linewidth=0.5)
ax_2.axes.yaxis.set_visible(False)


bumper = 0
shift = 0

for index in range(int((np.round(len(wavelength)/2)))):

    first = 2 * index
    second = (2 * index) + 1

    plot_1 = OD_norm[first][start[first]:stop[first]] + shift
    plot_2 = OD_norm[second][start[second]:stop[second]] + shift
    
    if bumper == index:

        bumper += 1

        ax_1.plot(wavelength[first][start[first]:stop[first]], plot_1, color=None, marker=None, linestyle='-', alpha=1, label=str(folder[first]))
        ax_2.plot(wavelength[second][start[second]:stop[second]], plot_2, color=None, marker=None, linestyle='-', alpha=1, label=str(folder[second]))

        shift += 0.05


ax_1.legend(loc='best', fontsize=8)
ax_2.legend(loc='best', fontsize=8)

In [ ]:
fig_2, ax_2 = mp.subplots(figsize=(8, 5))
ax_2.set_title('Pr$^{3+}$:YVO', fontsize=16)
ax_2.set_xlabel(xlabel='Wavelength (nm)', fontsize= 15)
ax_2.set_ylabel(ylabel='Absorption', fontsize=15)
sec_ax = ax_2.secondary_xaxis('top', functions= (lambda x: 1e7 / x, lambda x: 1e7 / x))
sec_ax.set_xlabel('Wavenumber (cm$^{-1}$)', fontsize=15)
ax_2.grid(True)
ax_2.grid(True, color='silver', linewidth=0.5)
ax_2.axes.yaxis.set_visible(False)
ax_2.tick_params(axis='both', which='major', labelsize=17)
sec_ax.tick_params(axis='both', which='major', labelsize=17)

x_coords = [6370, 6518, 6313]

plot_2 = OD_norm[1][start[1]:stop[1]] + 0.1
plot_3 = OD_norm[3][start[3]:stop[3]] + 0.15
plot_4 = OD_norm[5][start[5]:stop[5]] + 0.2
plot_1 = OD_norm[7][start[7]:stop[7]] + 0.05
plot_5 = OD_norm[9][start[9]:stop[9]] + 0.25

ax_2.plot(wavelength[7][start[7]:stop[7]], plot_1, color=None, marker=None, linestyle='-', alpha=1, label=str(folder[7]))
ax_2.plot(wavelength[1][start[1]:stop[1]], plot_2, color=None, marker=None, linestyle='-', alpha=1, label=str(folder[1]))
ax_2.plot(wavelength[3][start[3]:stop[3]], plot_3, color=None, marker=None, linestyle='-', alpha=1, label=str(folder[3]))
ax_2.plot(wavelength[5][start[5]:stop[5]], plot_4, color=None, marker=None, linestyle='-', alpha=1, label=str(folder[5]))
ax_2.plot(wavelength[9][start[9]:stop[9]], plot_5, color=None, marker=None, linestyle='-', alpha=1, label=str(folder[9]))

for data in x_coords:

    ax_2.axvline(x=1e7/data, linestyle='--', alpha=0.5)

ax_2.legend(loc='best', fontsize=14)